install bs4

In [1]:
pip install requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



def scrape_page(url,headers):
    # Step 1: Send a request to the webpage

    response = requests.get(url,headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
        # Step 2: Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # Step 3: Extract data
        id = [p.text for p in soup.find_all(attrs={'class': 'cusField p archive_no'}) if p.text != '档号']
        eid = [p.text for p in soup.find_all(attrs={'class': 'cusField p archive_eid'}) if p.text != '电子档号']
        titles = [p.text for p in soup.find_all(attrs={'class': 'cusField p title'}) if p.text != '题名']
        names_a = [p.text for p in soup.find_all(attrs={'class': 'cusField p official_name_a'}) if p.text != '责任者A']
        names_b = [p.text for p in soup.find_all(attrs={'class': 'cusField p official_name_b'}) if p.text != '责任者B']
        pos_a = [p.text for p in soup.find_all(attrs={'class': 'cusField p official_position_a'}) if p.text != '官职爵位A']
        pos_b = [p.text for p in soup.find_all(attrs={'class': 'cusField p official_position_b'}) if p.text != '官职爵位B']
        old_choro = [p.text for p in soup.find_all(attrs={'class': 'cusField p old_chronology'}) if p.text != '原纪年']
        return id,eid,titles,names_a,names_b,pos_a,pos_b,old_choro
    else:
        print(f'Failed to retrieve the webpage. Status code: {response.status_code}')
        return []
base_url = 'https://fhac.com.cn/ess/catalogue.html?0.6440211435717598&tpl_file=search_catalogues&pagesize=50&category_type=catalogue1&is_filter=1&filters%5B1%5D%5Bfilter_rel%5D=0&filters%5B1%5D%5Bfilter_field%5D=title&filters%5B1%5D%5Bfilter_type%5D=0&filters%5B1%5D%5Bfilter_kw%5D=%E8%9D%97&filters%5B2%5D%5Bfilter_rel%5D=1&filters%5B2%5D%5Bfilter_field%5D=title&filters%5B2%5D%5Bfilter_type%5D=0&filters%5B2%5D%5Bfilter_kw%5D=%E8%9D%BB&catalog%5B%5D=03&catalog%5B%5D=04&p='
num_pages = 43
headers = {
    'X-Requested-With': 'XMLHttpRequest'
}

all_id = []
all_eid = []
all_titles = []
all_names_a =[]
all_names_b =[]
all_pos_a = []
all_pos_b = []
all_old_choro =[]

for i in range(1,num_pages+1):
    url = f'{base_url}{i}'
    id,eid,titles,names_a,names_b,pos_a,pos_b,old_choro = scrape_page(url, headers)
    all_id.extend(id)
    all_eid.extend(eid)
    all_titles.extend(titles)
    all_names_a.extend(names_a)
    all_pos_a.extend(pos_a)
    all_names_b.extend(names_b)
    all_pos_b.extend(pos_b)
    all_old_choro.extend(old_choro)



# Step 4: Create a DataFrame
data = {
    'id': all_id,
    'eid': all_eid,
    'title': all_titles,
    'name_a': all_names_a,
    'pos_a': all_pos_a,
    'name_b': all_names_b,
    'pos_b': all_pos_b,
    'old_choro': all_old_choro
}

df = pd.DataFrame(data)

file_path = '/Users/henryma/Dropbox/Research/Locust_Disaster/Data/tiben_nan.xlsx'
df.to_excel(file_path, index=False)
print('Data has been written')


Data has been written
